### Learning Multivariate Shapelets with Multi-Layer Neural Networks
LTS(Learning Time-series Shapelets)では分類にロジスティクス回帰を用いるが、それをディープニューラルネットに置き換える。

#### 参考
①Roberto Medico, Joeri Ruyssinck, Dirk Deschrijver, Tom Dhaene.
Learning Multivariate Shapelets with Multi-Layer Neural Networks.
Advanced Course on Data Science & Machine Learning (ACDL) 2018.<br>
②Grabocka, J., Schilling, N., Wistuba, M. and SchmidtThieme, L.: Learning Time-series Shapelets, Proceedings of the 20th ACM SIGKDD International Conference on Knowledge Discovery and Data Mining, KDD
’14, ACM, pp. 392–401 (2014).


In [0]:
#使用するモジュール
from keras.models import Model
from keras import regularizers
from keras.layers import Dense, Conv1D, Layer, Input, BatchNormalization, Dropout
from keras.metrics import categorical_accuracy, categorical_crossentropy
from keras.utils import to_categorical
from keras.optimizers import Adam
import keras.backend as K
from keras.engine import InputSpec
import numpy as np
from scipy.io import arff
import matplotlib.pyplot as plt

from keras.initializers import Initializer
from tslearn.clustering import TimeSeriesKMeans
from tslearn.utils import to_time_series_dataset

### データを読み込み（事前にデータセットをカレントディレクトリに置くこと）

In [0]:
train_dataset, train_meta = arff.loadarff("ECGFiveDays_TRAIN.arff")
test_dataset, test_meta = arff.loadarff("ECGFiveDays_TEST.arff")

trds=np.asarray(train_dataset.tolist(), dtype=np.float32)
Len_data = trds.shape[1]

y_train=np.asarray(trds[:,Len_data-1].tolist(), dtype=np.int16)
X_train=trds[:, :Len_data-1]
teds=np.asarray(test_dataset.tolist(), dtype=np.float32)
y_test=np.asarray(teds[:,Len_data-1].tolist(), dtype=np.int16)
X_test=teds[:, :Len_data-1]

num_classes = len(set(y_train))

y_train_c = to_categorical(y_train-1, num_classes)
y_test_c = to_categorical(y_test-1, num_classes)

In [0]:
#正規化
from tslearn.preprocessing import TimeSeriesScalerMinMax
X_train_norm = TimeSeriesScalerMinMax().fit_transform(X_train)
X_test_norm = TimeSeriesScalerMinMax().fit_transform(X_test)

### カスタムレイヤーおよびカスタム初期化関数定義

In [0]:
class GlobalMinPooling1D(Layer):
    """Global min pooling operation for temporal data.
    # Input shape
        3D tensor with shape: `(batch_size, steps, features)`.
    # Output shape
        2D tensor with shape:
        `(batch_size, features)`
    """

    def __init__(self, **kwargs):
        super(GlobalMinPooling1D, self).__init__(**kwargs)
        self.input_spec = InputSpec(ndim=3)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[2]

    def call(self, inputs, **kwargs):
        return K.min(inputs, axis=1)


class LocalSquaredDistanceLayer(Layer):
    """Pairwise (squared) distance computation between local patches and shapelets
    # Input shape
        3D tensor with shape: `(batch_size, steps, features)`.
    # Output shape
        3D tensor with shape:
        `(batch_size, steps, n_shapelets)`
    """
    def __init__(self, n_shapelets, X, **kwargs):
        self.n_shapelets = n_shapelets
        self.initializer = KMeansShapeletInitializer(X)
        super(LocalSquaredDistanceLayer, self).__init__(**kwargs)
        self.input_spec = InputSpec(ndim=3)

    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel',
                                      shape=(self.n_shapelets, input_shape[2]),
                                      initializer='uniform',
                                      trainable=True)
        super(LocalSquaredDistanceLayer, self).build(input_shape)

    def call(self, x, **kwargs):
        # (x - y)^2 = x^2 + y^2 - 2 * x * y
        x_sq = K.expand_dims(K.sum(x ** 2, axis=2), axis=-1)
        y_sq = K.reshape(K.sum(self.kernel ** 2, axis=1), (1, 1, self.n_shapelets))
        xy = K.dot(x, K.transpose(self.kernel))
        return x_sq + y_sq - 2 * xy

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[1], self.n_shapelets
      
      
      
def _kmeans_init_shapelets(X, n_shapelets, shp_len, n_draw=10000):
    n_ts, sz, d = X.shape
    indices_ts = numpy.random.choice(n_ts, size=n_draw, replace=True)
    indices_time = numpy.random.choice(sz - shp_len + 1, size=n_draw, replace=True)
    subseries = numpy.zeros((n_draw, shp_len, d))
    for i in range(n_draw):
        subseries[i] = X[indices_ts[i], indices_time[i]:indices_time[i] + shp_len]
    return TimeSeriesKMeans(n_clusters=n_shapelets,
                            metric="euclidean",
                            verbose=False).fit(subseries).cluster_centers_


class KMeansShapeletInitializer(Initializer):
    """Initializer that generates shapelet tensors based on a clustering of time series snippets.
    # Arguments
        dataset: a dataset of time series.
    """
    def __init__(self, X):
        self.X_ = to_time_series_dataset(X)

    def __call__(self, shape, dtype=None):
        n_shapelets, shp_len = shape
        shapelets = _kmeans_init_shapelets(self.X_,
                                           n_shapelets,
                                           shp_len)[:, :, 0]
        return K.tensorflow_backend._to_tensor(x=shapelets, dtype=K.floatx())

    def get_config(self):
        return {'data': self.X_}

def init_I(shape, dtype=None):
    weights_I = np.empty(shape,dtype=dtype)
    for di in range(shape[1]):
      weights_I[:, di, :] = np.eye(shape[2])
    return weights_I

### MLNN_shapeletsの実装

In [0]:
inputshape = X_train_norm.shape

sz = 20 #shapeletsの長さ
num_shapelet = 15 #shapeletsの数
input_shape = X_train_norm.shape #入力データのシェイプ

inputs = Input(shape=(input_shape[1], input_shape[2]), name="input")

#以下の3層が1つのDistanceLayer
x = Conv1D(filters=sz,
           kernel_size=sz,
           kernel_initializer=init_I,
           trainable=False,
           use_bias=False,
           name="DistanceLayer_1")(inputs)
x = LocalSquaredDistanceLayer(num_shapelet,X=X_test_norm,name="DistanceLayer_2")(x)
x = GlobalMinPooling1D(name="DistanceLayer_3")(x)

x = BatchNormalization()(x)
x = Dense(30, activation='relu', name = "hidden_1")(x)

x = BatchNormalization()(x)
x = Dropout(0.5)(x)
outputs = Dense(num_classes, activation='softmax',kernel_regularizer=regularizers.l2(0.01), name = "output")(x)

s_model = Model(inputs=inputs, outputs=outputs)

In [144]:
s_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 136, 1)            0         
_________________________________________________________________
DistanceLayer_1 (Conv1D)     (None, 117, 20)           400       
_________________________________________________________________
DistanceLayer_2 (LocalSquare (None, 117, 15)           300       
_________________________________________________________________
DistanceLayer_3 (GlobalMinPo (None, 15)                0         
_________________________________________________________________
batch_normalization_11 (Batc (None, 15)                60        
_________________________________________________________________
hidden_1 (Dense)             (None, 30)                480       
_________________________________________________________________
batch_normalization_12 (Batc (None, 30)                120       
__________

DistanceLayerは3つまとめて論文のDistanceLayer。
1層目で入力データをスライディング窓サイズ=shapelets_lengthでずらし続けたものを並べて行列として出力。2層目は各shapeletに対する距離Dを計算。3層目でそれらのミニマムを取り、ニューラルネットへの入力であるMを得る。
以後は普通のDNN。

### モデルのコンパイルと学習

In [0]:
s_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
s_model.fit(X_train_norm,y_train_c,epochs=1000,validation_data=(X_test_norm,y_test_c), verbose=0)

In [154]:
print("**** TEST ****")
print(list(zip(s_model.metrics_names, s_model.evaluate(X_train_norm,y_train_c, verbose=0))))
print("**** TEST ****")
print(list(zip(s_model.metrics_names, s_model.evaluate(X_test_norm,y_test_c, verbose=0))))

**** TEST ****
[('loss', 0.00496688624843955), ('acc', 1.0)]
**** TEST ****
[('loss', 0.14262923211482345), ('acc', 0.9454123120274693)]


訓練データの精度は100%、テストデータの精度94%くらい。　


### 参考:普通のSVMでの精度

In [156]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler


clf = SVC(gamma='auto')
X_svm_trn = X_train.reshape(X_train.shape[0],X_train.shape[1])
X_svm_tst = X_test.reshape(X_test.shape[0],X_test.shape[1])
                                                      
scaler = StandardScaler()
scaler.fit(X_svm_trn)
                                                        
X_train_scaled = scaler.transform(X_svm_trn)
X_test_scaled = scaler.transform(X_svm_tst)
                                                      
clf.fit(X_train_scaled,y_train)
print(clf.score(X_train_scaled,y_train))
print(clf.score(X_test_scaled,y_test))

1.0
0.8397212543554007
